In [42]:
import numpy as np
import cv2

import glob

from os import path as ospath

CAMERA CALIBRATION FROM OPENCV SITE

In [43]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
 
images = glob.glob('images/chess-*.jpg')
 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
 
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imwrite(f'out/{ospath.basename(fname)}', img)


images\chess-0.jpg
images\chess-1.jpg
images\chess-10.jpg
images\chess-11.jpg
images\chess-2.jpg
images\chess-3.jpg
images\chess-4.jpg
images\chess-5.jpg
images\chess-6.jpg
images\chess-7.jpg
images\chess-8.jpg
images\chess-9.jpg


In [44]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [45]:
img = cv2.imread('images/chess-1.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [46]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

True

HEIGHT CALCULATION

In [82]:
def find_liquid_height(image, debug_name='out.jpg', glass_height_cm=10):
    # Carica l'immagine
    # image = cv2.imread(image_path)
    # if image is None:
    #     raise FileNotFoundError(f"Immagine non trovata: {image_path}")

    # Ridimensiona immagine per facilitare l'elaborazione (opzionale)
    scale_percent = 50  # riduci al 50% della dimensione originale
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    image = cv2.resize(image, (width, height))

    # Converti in scala di grigi
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    # gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # Rileva i bordi con Sobel
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient = cv2.magnitude(grad_x, grad_y)
    _, glass_edges = cv2.threshold(np.uint8(gradient), 30, 255, cv2.THRESH_BINARY)
    
    # Applica operazioni morfologiche per rafforzare i bordi del bicchiere
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    glass_edges = cv2.morphologyEx(glass_edges, cv2.MORPH_CLOSE, kernel)
    cv2.imwrite(f'out/glass-{debug_name}', glass_edges)

    # Trova i contorni nella maschera del bicchiere
    contours, _ = cv2.findContours(glass_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Trova il contorno più grande (supposto essere il bicchiere)
    if len(contours) == 0:
        raise ValueError("Bicchiere non rilevato.")
    glass_contour = max(contours, key=cv2.contourArea)

    # Determina il rettangolo che contiene il bicchiere
    x, y, w, h = cv2.boundingRect(glass_contour)

    # Altezza del bicchiere in pixel
    glass_height_px = h

    # Ritaglia l'immagine per concentrarsi sul bicchiere
    # roi = image[y:y+h, x:x+w]
    roi_gray = gray[y:y+h, x:x+w]

    # Rileva il liquido utilizzando soglia adattiva
    # liquid_mask = cv2.adaptiveThreshold(roi_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    # Usa la soglia di Otsu per segmentare il liquido
    _, liquid_mask = cv2.threshold(roi_gray, 100, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    cv2.imwrite(f'out/liquid-{debug_name}', liquid_mask)
    # Trova i contorni del liquido
    liquid_contours, _ = cv2.findContours(liquid_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Draw contours
    drawing = np.zeros((liquid_mask.shape[0], liquid_mask.shape[1], 3), dtype=np.uint8)
    for i in range(len(liquid_contours)):
        color = (np.random.randint(0,256), np.random.randint(0,256), np.random.randint(0,256))
        cv2.drawContours(drawing, liquid_contours, i, color, 2, cv2.LINE_8, 0)
    # Show in a window
    cv2.imwrite(f'out/c-liquid-{debug_name}', drawing)

    # Trova il contorno del liquido (ipotizzando che sia il più grande nella ROI)
    liquid_contour = max(liquid_contours, key=cv2.contourArea)

    # Determina il rettangolo che contiene il liquido
    _, liquid_y, _, liquid_h = cv2.boundingRect(liquid_contour)

    # Altezza del liquido in pixel
    liquid_height_px = liquid_h

    # Calcola il rapporto pixel/cm
    px_per_cm = glass_height_px / glass_height_cm

    # Altezza del liquido in cm
    liquid_height_cm = liquid_height_px / px_per_cm

    # Disegna i contorni per il debug
    debug_image = image.copy()
    cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Bicchiere
    cv2.rectangle(debug_image, (x, y + liquid_y), (x + w, y + liquid_y + liquid_h), (255, 0, 0), 2)  # Liquido

    # Mostra l'immagine per il debug
    width = int(debug_image.shape[1] * 0.25)
    height = int(debug_image.shape[0] * 0.25)
    debug_image = cv2.resize(debug_image, (width, height))
    cv2.imwrite(f'out/degub-{debug_name}', debug_image)

    return liquid_height_cm

# Percorso immagine
image_path = "images/A-A.jpg"

# Calcola l'altezza del liquido
try:
    image = cv2.imread(image_path)
    liquid_height = find_liquid_height(image)
    print(f"Altezza del liquido: {liquid_height:.2f} cm")
except Exception as e:
    print(f"Errore: {e}")


Altezza del liquido: 3.82 cm


FUSION

In [89]:
import re
from utils import get_glass_height_cm

filelist = glob.glob("images/?-?.jpg")

for file in filelist:
    # Percorso immagine
    tipe = re.search('([A-C]?)-[A-C]?', file).group(1)
    height = get_glass_height_cm(tipe)

    img = cv2.imread(file)
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    
    # Taglia il bordo del tavolo
    # xcrop = 500
    # ycrop = 150
    #dst = dst[y+ycrop:y+h-ycrop, x+xcrop:x+w-xcrop]
    cv2.imwrite(f'out/calib-{ospath.basename(file)}', dst)

    # Calcola l'altezza del liquido
    try:
        liquid_height = find_liquid_height(img, debug_name=ospath.basename(file), glass_height_cm=height)
        print(f"Altezza del liquido in {ospath.basename(file)}: {round(liquid_height*10, 0)} mm")
    except Exception as e:
        print(f"Errore: {e}")

Altezza del liquido in A-A.jpg: 37.0 mm
Altezza del liquido in A-C.jpg: 53.0 mm
Altezza del liquido in A-F.jpg: 28.0 mm
Altezza del liquido in B-A.jpg: 58.0 mm
Altezza del liquido in B-C.jpg: 59.0 mm
Altezza del liquido in B-F.jpg: 27.0 mm
Altezza del liquido in C-A.jpg: 53.0 mm
Altezza del liquido in C-C.jpg: 65.0 mm
Altezza del liquido in C-F.jpg: 37.0 mm
